In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from torch.utils.data import Dataset, DataLoader
import gc

In [14]:
gc.collect()
torch.cuda.empty_cache()

In [3]:
file_path = r"C:\Users\krish\Downloads\archive (7)\weatherHistory.csv"
df = pd.read_csv(file_path)

In [4]:
# Drop unnecessary columns
df = df.drop(columns=["Loud Cover"])

# Convert date column
df["Formatted Date"] = pd.to_datetime(df["Formatted Date"])
df = df.sort_values("Formatted Date")
df = df.drop(columns=["Formatted Date"])

# Fill missing values
df["Precip Type"].fillna("unknown", inplace=True)
df = pd.get_dummies(df, columns=["Precip Type"], drop_first=True)

# Encode textual features
label_encoder_summary = LabelEncoder()
df["Summary"] = label_encoder_summary.fit_transform(df["Summary"])

label_encoder_daily_summary = LabelEncoder()
df["Daily Summary"] = label_encoder_daily_summary.fit_transform(df["Daily Summary"])

# Normalize data
scaler = MinMaxScaler()
data = scaler.fit_transform(df.values)

# Create sequences
def create_sequences(data, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i+seq_length])
        targets.append(data[i+seq_length])
    return np.array(sequences), np.array(targets)

seq_length = 24  # Using past 24 hours to predict next timestep
X, y = create_sequences(data, seq_length)

C:\Users\krish\AppData\Local\Temp\ipykernel_34272\1136123329.py:5: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df["Formatted Date"] = pd.to_datetime(df["Formatted Date"])
C:\Users\krish\AppData\Local\Temp\ipykernel_34272\1136123329.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inpla

In [5]:
# Convert to PyTorch tensors
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, y_train = torch.tensor(X, dtype=torch.float32).to(device), torch.tensor(y, dtype=torch.float32).to(device)

In [6]:
# Define dataset class
class WeatherDataset(Dataset):
    def __init__(self, X, y):
        self.X, self.y = X, y
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create dataloader
dataset = WeatherDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [7]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :])

In [8]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

In [9]:
class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BiLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

In [10]:
class StackedLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2):
        super(StackedLSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])

In [11]:
# Training function
def train_model(model, dataloader, epochs=10, lr=0.001):
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        for X_batch, y_batch in dataloader:
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
    return model

In [12]:
# Train all models
input_size = X_train.shape[2]
hidden_size = 64
output_size = y_train.shape[1]

models = {
    "RNN": RNNModel(input_size, hidden_size, output_size),
    "LSTM": LSTMModel(input_size, hidden_size, output_size),
    "BiLSTM": BiLSTMModel(input_size, hidden_size, output_size),
    "StackedLSTM": StackedLSTMModel(input_size, hidden_size, output_size)
}

for name, model in models.items():
    print(f"Training {name}...")
    models[name] = train_model(model, dataloader)


Training RNN...
Epoch 1, Loss: 0.0086
Epoch 2, Loss: 0.0081
Epoch 3, Loss: 0.0103
Epoch 4, Loss: 0.0098
Epoch 5, Loss: 0.0111
Epoch 6, Loss: 0.0190
Epoch 7, Loss: 0.0086
Epoch 8, Loss: 0.0120
Epoch 9, Loss: 0.0120
Epoch 10, Loss: 0.0099
Training LSTM...
Epoch 1, Loss: 0.0103
Epoch 2, Loss: 0.0107
Epoch 3, Loss: 0.0092
Epoch 4, Loss: 0.0079
Epoch 5, Loss: 0.0076
Epoch 6, Loss: 0.0071
Epoch 7, Loss: 0.0131
Epoch 8, Loss: 0.0059
Epoch 9, Loss: 0.0063
Epoch 10, Loss: 0.0125
Training BiLSTM...
Epoch 1, Loss: 0.0088
Epoch 2, Loss: 0.0129
Epoch 3, Loss: 0.0145
Epoch 4, Loss: 0.0107
Epoch 5, Loss: 0.0112
Epoch 6, Loss: 0.0076
Epoch 7, Loss: 0.0102
Epoch 8, Loss: 0.0091
Epoch 9, Loss: 0.0107
Epoch 10, Loss: 0.0126
Training StackedLSTM...
Epoch 1, Loss: 0.0113
Epoch 2, Loss: 0.0071
Epoch 3, Loss: 0.0120
Epoch 4, Loss: 0.0080
Epoch 5, Loss: 0.0096
Epoch 6, Loss: 0.0109
Epoch 7, Loss: 0.0066
Epoch 8, Loss: 0.0093
Epoch 9, Loss: 0.0079
Epoch 10, Loss: 0.0085


In [13]:
# Evaluate models with batch processing to avoid OOM error
def evaluate_model(model, dataloader):
    model.eval()
    total_loss = 0.0
    criterion = nn.MSELoss()
    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            total_loss += loss.item()
    
    return total_loss / len(dataloader)

# Evaluate each model in batches
for name, model in models.items():
    mse = evaluate_model(model, dataloader)
    print(f"{name} MSE: {mse:.4f}")

RNN MSE: 0.0104
LSTM MSE: 0.0097
BiLSTM MSE: 0.0098
StackedLSTM MSE: 0.0096
